# LocalOutlierFactor
Local Outlier Factor (LOF) основывается на концепции поиска ближайшего соседа и рассчитывает локальное отклонение плотности точки по отношению к ее соседям. LOF используется для обнаружения выбросов и «новизны» - объектов, которые отличаются от других объектов выборки.

LOF оценивает плотность распределения объектов выборки. Если точка имеет мало ближайших соседей, это означает, что она находится в области с малой плотностью, что делает ее потенциальной аномалией. Если точка имеет много ближайших соседей, это означает, что она находится в области с высокой плотностью, и, следовательно, не является аномалией.

По умолчанию LOF предназначен только для обнаружения выбросов (novelty=False). Установив значение novelty в True, можно использовать LOF для обнаружения новизны. В этом случае следует помнить, что использовать функции predict, decision_function и score_samples можно только на новых невидимых данных, а не на обучающем множестве, а результаты, полученные таким образом, могут отличаться от стандартных результатов LOF.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# ! cp -r /content/drive/MyDrive/Study/MIPT_magistery/qualification_work/data .

In [3]:
import os
import random
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# ml_semisupervised_methods
from sklearn.neighbors import LocalOutlierFactor


# metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def seed_everything(seed):
    # фискирует максимум сидов для корректности сравнения разных экспериментов
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
SEED = 42
seed_everything(SEED)

# Datasets

In [4]:
# Синтетически сгенерированные нормально распределенные данные
def make_norm_data(rows, columns, noise_percent=0.05):
    data_norm = pd.DataFrame(data=np.random.normal(
                            loc=0, scale=1, size=(rows, columns)))
    noise = pd.DataFrame(data=np.random.uniform(
                            low=-6, high=6, size=(int(data_norm.shape[0]*noise_percent), data_norm.shape[1])))

    data_noise = pd.DataFrame()
    for feature in noise.columns:
        filter = (
            (noise[feature] < data_norm[feature].min())
            | (noise[feature] > data_norm[feature].max())
            )
        data_noise = pd.concat([data_noise, noise[filter]])
    data_noise = data_noise.drop_duplicates()
    data_norm['anomaly'] = 0
    data_noise['anomaly'] = 1

    df_norm = pd.concat((data_norm, data_noise))
    return df_norm

df_norm = make_norm_data(1000, 8)
df_norm

,0,1,2,3,4,5,6,7,anomaly
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,0
1,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,-0.562288,0
2,-1.012831,0.314247,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748,0
3,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,0
4,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.959670,-1.328186,0.196861,0
...,...,...,...,...,...,...,...,...,...
32,0.063984,1.772509,0.031678,-0.098264,-3.938531,-2.655435,-4.093768,-3.652004,1
38,-2.800126,2.593122,-1.268901,2.124984,-0.288220,5.925277,2.746826,5.429896,1
5,2.151800,-0.353991,-1.345218,-1.488481,1.680586,-0.062468,4.189587,4.496424,1
37,-2.554173,0.008819,-0.528646,-1.866863,2.324896,3.114742,4.235808,5.674279,1


In [5]:
# SKAB data

all_files=[]
for root, dirs, files in os.walk("data/skab/"):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))

# формируем датафрейм
dfs=[]
for path in all_files:
    df = pd.read_csv(path,index_col='datetime',sep=';',parse_dates=True)
    # print(path, df.shape)
    dfs.append(df)
# print('Features:')
# for col in dfs[2].columns:
#     print('\t',col)
dfs = [df for df in dfs if df.shape[1] == 10]
df_skab = pd.concat(dfs)
# print(df_skab.shape)
df_skab = df_skab.drop_duplicates()
df_skab = df_skab.drop('changepoint', axis=1).sort_index()
display(df_skab)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly
datetime,,,,,,,,,
2020-03-09 14:54:41,0.027078,0.039949,1.029940,0.054711,66.0458,24.5843,237.477,32.9719,0.0
2020-03-09 14:54:42,0.027346,0.039636,1.194400,-0.601143,66.0547,24.5866,243.741,32.0000,0.0
2020-03-09 14:54:43,0.027374,0.039181,0.697448,0.054711,66.0166,24.5821,230.858,32.0000,0.0
2020-03-09 14:54:44,0.027761,0.040159,0.633259,0.054711,66.1645,24.5788,209.755,32.0000,0.0
2020-03-09 14:54:46,0.027564,0.040767,1.520800,0.054711,66.1310,24.5746,247.676,32.0290,0.0
...,...,...,...,...,...,...,...,...,...
2020-03-01 17:00:49,0.080358,0.136703,2.006250,0.054711,85.4852,22.0975,224.104,75.0206,1.0
2020-03-01 17:00:50,0.081409,0.129041,1.789790,0.054711,85.5540,22.0898,230.848,75.9796,1.0
2020-03-01 17:00:51,0.081189,0.132478,2.117070,0.054711,86.1248,22.1004,248.304,75.0000,1.0


In [6]:
datasets = {
    'df_norm': df_norm,
    'df_skab': df_skab
}

# Метрики

In [7]:
def score_metrics(real_outliers, pred_outliers):
    scores = {}
    scores = classification_report(real_outliers, pred_outliers)
    return scores

# Preprocessing

In [8]:
def preprocessing(df):
    X = df.copy()
    y = X.pop('anomaly')

    # preprocessing
    columns = list(X.columns)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(data=X, columns=columns)
    return X, y

# LocalOutlierFactor

In [10]:
for df_name, df in datasets.items():
    print(df_name)
    df_skab_cleaned = df[df['anomaly'] == 0]
    df_skab_anomaly = df[df['anomaly'] == 1]

    df_train, df_test = train_test_split(
                df_skab_cleaned, test_size=0.5, random_state=SEED)
    df_test = pd.concat([df_test, df_skab_anomaly])
    df_test = shuffle(df_test)

    X_train, _ = preprocessing(df_train)
    X_test, y_test = preprocessing(df_test)
    lof = LocalOutlierFactor(novelty=True)
    lof.fit(X_train.values)
    pred = lof.predict(X_test.values)
    pred[pred != -1] = 0
    pred[pred == -1] = 1
    print(score_metrics(y_test, pred))

df_norm
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      0.96      0.98        50

    accuracy                           1.00       550
   macro avg       1.00      0.98      0.99       550
weighted avg       1.00      1.00      1.00       550

df_skab
              precision    recall  f1-score   support

         0.0       0.48      0.68      0.56     11927
         1.0       0.53      0.34      0.42     13067

    accuracy                           0.50     24994
   macro avg       0.51      0.51      0.49     24994
weighted avg       0.51      0.50      0.49     24994

